This analysis looks at burn severity of wildfires in the Sierra Nevada region from 2012 to 2021. 

 
- Not all wildfires were documented in the CBI-4 data sets
- The size here is calculated by numberOfCell * x-resolution * y-resolution. >> sq. meters
- The index is used to classify burn severity as unchanged, low, moderate or high severity. >> impacts not intensity
    - Unchanged: Areas indistinguishable from pre-fire conditions. This does not necessarily indicate that the area was unburned. 
    - Low: Areas of surface fire with little change in cover and little mortality of the structurally dominant vegetation. 
    - Moderate: Areas between low and high severity, with a mixture of effects on the structurally dominant vegetation. 
    - High: Areas where the dominant vegetation has high to complete mortality. 

refs: calculating tif class area size
- https://gis.stackexchange.com/questions/199113/how-to-calculate-raster-area-or-number-of-cells-for-multiple-classes-in-a-raster?noredirect=1&lq=1
- https://gis.stackexchange.com/questions/277126/what-are-the-methods-for-calculating-area-from-a-raster-in-qgis

Questions
- How much has the area burned at the highest severity increased since 2012? 
- Has % of area burned at the highest severity also increased? 
- New areas burned at highest severity?
- Impacts of the increasing areas with high burn index? >> satellite imgs?
    - tree canopy?
    - animal habitats? 
    - property loss? 
    - ...
- Composite Burn Index (CBI) Data for the Conterminous US, [Collected Between 1996 and 2018](https://www.sciencebase.gov/catalog/item/5d963705e4b0c4f70d110ee6)

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from osgeo import gdal
import rasterio
import altair as alt
import altair_latimes as lat

In [5]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

In [7]:
def analyze_index(file):
    raw = pd.read_csv(file, index_col = 0)
    raw['percent_of_total'] = 0
    df = raw.copy()
    grouped = df.groupby('year').sqm.sum().reset_index()
    for i in range(len(grouped)):
        for k in range(len(df)):
            if grouped.year[i] == df.year[k]:
                df['percent_of_total'].iloc[k] = df.sqm[k] / grouped.sqm[i]
    return grouped, df

# CBI-4 (burn severity index)

In [53]:
cbi4_by_year, cbi4_by_index = analyze_index('output/sum_of_sierra_fire_size_by_burn_index.csv')

/var/folders/zv/3yvy_fbd0mz2lvqqtd13wypc0000gp/T/ipykernel_7371/2626247015.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['percent_of_total'].iloc[k] = df.sqm[k] / grouped.sqm[i]


In [54]:
alt.Chart(cbi4_by_year, title="Total area burned by year").mark_bar(size=20).encode(
    x=alt.X(
        "year",
        title="Year",
        axis=alt.Axis(format=''),
        scale=alt.Scale(domain=[2012, 2021])
    ),
    y=alt.Y(
        'sqm',
        title='Sq. meters',
        axis=alt.Axis(format=""),
    ),
    tooltip=['year', 'sqm']
)

alt.Chart(...)

In [55]:
total_burn_after20 = cbi4_by_year.iloc[8:].sqm.sum()
total_burn_before20 = cbi4_by_year.iloc[:8].sqm.sum()

In [56]:
total_burn_after20 / total_burn_before20

1.6020812986433912

- The total area burned after 2020 was 1.6 times the total area burned from 2012 to 2019. 
- The total area burned in 2021 was 10.6 times that of 2012.

In [58]:
high_severity = cbi4_by_index[cbi4_by_index.ind == 4].reset_index(drop=True)

alt.Chart(high_severity, title="Total area burned at the highest severity by year").mark_bar(size=20).encode(
    x=alt.X(
        "year",
        title="Year",
        axis=alt.Axis(format=''),
        scale=alt.Scale(domain=[2012, 2021])
    ),
    y=alt.Y(
        'sqm',
        title='Sq. meters',
        axis=alt.Axis(format=""),
    ),
    tooltip=['year', 'sqm']
)

alt.Chart(...)

In [23]:
hsevere_after20 = high_severity.iloc[8:].sqm.sum()
hsevere_before20 = high_severity.iloc[:8].sqm.sum()

In [24]:
hsevere_after20 / hsevere_before20

2.202554695513226

- The total area burned at high severity after 2020 was 2.2 times that of 2012 to 2019.
- In 2021, the total area burned at the highest severity level was 22 times that of 2012.

In [62]:
alt.Chart(high_severity, title="Percentage of total area burned at the highest severity by year").mark_bar(size=20).encode(
    x=alt.X(
        "year",
        title="Year",
        axis=alt.Axis(format=''),
        scale=alt.Scale(domain=[2012, 2021])
    ),
    y=alt.Y(
        'percent_of_total',
        title='',
        axis=alt.Axis(format=".0%"),
    ),
    tooltip=['year', 'percent_of_total']
)

alt.Chart(...)

The % of fire burned at the highest severity in Sierra Nevada peaked in 2016.

# Basal area change

In [64]:
ba_by_year, ba_by_index = analyze_index('output/sum_of_sierra_area_by_ba_index.csv')

/var/folders/zv/3yvy_fbd0mz2lvqqtd13wypc0000gp/T/ipykernel_7371/2626247015.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['percent_of_total'].iloc[k] = df.sqm[k] / grouped.sqm[i]


In [65]:
ba_loss_ninety = ba_by_index[ba_by_index.ind == 7].reset_index(drop=True)

alt.Chart(ba_loss_ninety, title="Areas with more than 90% basal loss by year, sq. mt").mark_bar(size=20).encode(
    x=alt.X(
        "year",
        title="Year",
        axis=alt.Axis(format=''),
        scale=alt.Scale(domain=[2012, 2021])
    ),
    y=alt.Y(
        'sqm',
        title='Sq. meters',
        axis=alt.Axis(format=""),
    ),
    tooltip=['year', 'sqm']
)

alt.Chart(...)

Wildfire land that had lost more than 90% of its basal area totaled 2.95 billion square meters in 2021.

In [100]:
ba_loss_ninety

,year,ind,ind_name,sqm,percent_of_total
0,2012,7,>=90%,1.371204e+08,0.213361
1,2013,7,>=90%,4.443687e+08,0.338014
2,2014,7,>=90%,4.134159e+08,0.479243
3,2015,7,>=90%,2.806047e+08,0.394742
4,2016,7,>=90%,2.152008e+08,0.515182
5,2017,7,>=90%,1.780335e+08,0.295806
6,2018,7,>=90%,5.130252e+08,0.245908
7,2019,7,>=90%,1.333332e+08,0.379627
8,2020,7,>=90%,2.073737e+09,0.473462
9,2021,7,>=90%,2.950955e+09,0.432998


# Canopy coverage loss

In [66]:
cc_by_year, cc_by_index = analyze_index('output/sum_of_sierra_area_by_cc_index.csv')

/var/folders/zv/3yvy_fbd0mz2lvqqtd13wypc0000gp/T/ipykernel_7371/2626247015.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['percent_of_total'].iloc[k] = df.sqm[k] / grouped.sqm[i]


In [67]:
cc_loss_svnfv = cc_by_index[cc_by_index.ind == 5].reset_index(drop=True)

alt.Chart(cc_loss_svnfv, title="Areas with more than 75% canopy coverage loss by year, sq. mt").mark_bar(size=20).encode(
    x=alt.X(
        "year",
        title="Year",
        axis=alt.Axis(format=''),
        scale=alt.Scale(domain=[2012, 2021])
    ),
    y=alt.Y(
        'sqm',
        title='Sq. meters',
        axis=alt.Axis(format=""),
    ),
    tooltip=['year', 'sqm']
)

alt.Chart(...)

3.36 billion square meters of wildfire land lost more than 75% of its canopy coverage in 2021.

In [101]:
cc_loss_svnfv

,year,ind,ind_name,sqm,percent_of_total
0,2012,5,>=75%,1.635840e+08,0.254539
1,2013,5,>=75%,5.071770e+08,0.385790
2,2014,5,>=75%,4.557366e+08,0.528303
3,2015,5,>=75%,3.223656e+08,0.453490
4,2016,5,>=75%,2.377584e+08,0.569184
5,2017,5,>=75%,2.138328e+08,0.355287
6,2018,5,>=75%,6.124914e+08,0.293585
7,2019,5,>=75%,1.511622e+08,0.430390
8,2020,5,>=75%,2.283600e+09,0.521376
9,2021,5,>=75%,3.306032e+09,0.485099


In [28]:
# def read_geotiff(file):
#     df = rasterio.open(file)
#     return df

# tif_ls = []

# for i in range(10):
#     year = 2012+i
#     tif = read_geotiff('output/tif_file_by_index/'+str(year)+'_sierra_clipped_high.tif')
#     tif_ls.append({'year': year, 'img':tif})


In [32]:
# def show(imgs):
#     '''Show in Jupyter notebook one or sequence of PIL images in a row.'''
#     figsize = [15,10]
#     row = 2
#     col = int(len(imgs)/row)
#     fig, ax = plt.subplots(row, col,figsize=figsize)
#     if len(imgs)==1:
#         ax=[ax]
    
#     for num,el in enumerate(imgs):
#         if (num < col):
#             nrol = 0
#         else:
#             nrol = 1
#         ax[nrol,int(num%col)].set_title(el['year'])
#         ax[nrol,int(num%col)].imshow(el['img'])
#         ax[nrol,int(num%col)].axis('off')
    
# show(tif_ls)